<a href="https://colab.research.google.com/github/Murcha1990/ML_AI25/blob/main/Hometasks/Base/AI_HW3_Classification_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание 3. Линейная классификация (base)**

### **Оценивание и штрафы**

С наступающим новым годом, друзья! Магистратура бежит быстро и мы бежим очень быстро, а зима — то время, когда хотелось бы бежать чуть медленнее. Поэтому это домашнее задание мы сделали сильно короче от его начальной версии!

Как всегда, каждая из задач имеет «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

В задании две части:

- Часть 1: написание логистической регрессии своими руками — уверенны, логлосс вы уже знаете как свои пять пальцев.
- Часть 2: решение задачи классификации на текстах.

In [ ]:
%pylab inline
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC


from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
#from google.colab import drive # Если вы работаете в коллабе
#drive.mount('/content/drive')

# **Часть 1. Логистическая регрессия своими руками (5 баллов)**

Логистическая регрессия — безумно важная и удобная модель для понимания начальных концепций. Вы много практиковались с выведением формулы градиента логлосса, шага спуска, а в прошлом дз сделали SGD. Давайте сделаем ещё шаг вперед — и реализуем логистическую регрессию своими руками.

На практике, часто хватает алгоритмов из коробки. Но иногда очень удобно сделать свой алгоритм.

## **Теоретическая сноска: почему LogLoss'а так много**

Почти наверное (в математике это значит, во всех случаях, кроме множества размером 0) логлосс набил оскомину за несколько заданий. Давайте посмотрим на него ещё раз:

В логистической регрессии функция потерь

$$\text{LogLoss}(y, \hat{p}) = -\left(y\log \hat{p} + (1-y)\log (1-\hat{p})\right)$$

Зачем мы так долго с ней возимся?

#### **Пункт 1.**
Во-первых, это **следствие максимизации правдоподобия** при биномиальной модели.

Если считать, что целевая переменная (Y\in{0,1}) распределена как

$$P(Y=1 \mid x) = \hat{p}(x), \qquad P(Y=0\mid x) = 1-\hat{p}(x),$$
то правдоподобие выборки ( (x_i, y_i) )\ равно
$$L = \prod_{i=1}^n \hat{p}_i^{y_i}(1-\hat{p}_i)^{1-y_i}.$$

Максимизация $\log L$ эквивалентна минимизации LogLoss.
Таким образом, LogLoss — **единственная функция потерь, полностью согласованная с вероятностной моделью логистической регрессии**.

#### **Пункт 2.**

Во-вторых, логлосс поможет нам в будущем понять другие функции потерь. Так, например LogLoss является частным случаем **кросс-энтропии между истинным распределением и предсказанным**.

Для двух распределений $p$ (истинного) и $q$ (предсказанного) кросс-энтропия определяется как

$$H(p,q) = -\sum_{k} p(k)\log q(k).$$

В бинарном случае истинное распределение дискретно:

$$p = (y, 1-y), \qquad q = (\hat{p}, 1-\hat{p}),$$
и подстановка даёт

$$H(p,q) = -\left[y\log \hat{p} + (1-y)\log (1-\hat{p})\right] = \text{LogLoss}.$$


В общем, любим, жалуем и реализуем.




### **Задание 1. Реализуйте класс логистической регрессии, обучаемой с помощью:**

**Задание 1.1 (1.5 балла). Градиентного спуска**

**Задание 1.2 (1.5 балла). Стохастического градиентного спуска**

До этого вы писали код без ограничений. Здесь же необходимо соблюдать следующие условия:

- Градиентный спуск необходимо записать в векторном виде;
- Циклы средствами python допускается использовать только для итераций градиентного спуска;

**Класс градиентного спуска должен:**
- В качестве критерия останова использовать (одновременно):
  - проверку на евклидову норму разности весов на двух соседних итерациях задаваемого параметром `tolerance`;
  - достижение максимального числа итераций, задаваемого параметром `max_iter`.
- Обладать атрибутом `loss_history`. В нём после вызова метода fit должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту). Данный атрибут необходим, чтобы проследить, что оптимизационный процесс действительно сходится;
- Инициализировать веса случайным образом или нулевым вектором (на ваш выбор).

Полезно [почитать](https://scikit-learn.org/stable/developers/develop.html).

**Шаблон класса описан ниже, вам нужно реализовать каждую из заготовленных функций.**

**ВАЖНО!** Мы заполняем данный шаблон, даже если он нам не нравится. Менять структуру класса и писать по-своему запрещено - за это будут сняты баллы.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator

class LogReg(BaseEstimator):
    def __init__(self, gd_type='stochastic',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2):
        """
        gd_type: 'full' or 'stochastic'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) — init weights
        eta: learning rate
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.w = None
        self.eta = eta
        self.loss_history = []

    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        ell, d = X.shape

        # Инициализация весов
        if self.w0 is not None:
            self.w = self.w0.copy()
        else:
            self.w = np.zeros(d)

        self.loss_history = []

        for _ in range(self.max_iter):
            # Записываем loss до обновления весов
            self.loss_history.append(self.calc_loss(X, y))

            # Вычисляем градиент
            if self.gd_type == 'full':
                grad = self.calc_gradient(X, y)
            else:  # стохастический градиент
                idx = np.random.randint(0, ell)
                grad = self.calc_gradient(X[idx:idx+1], y[idx:idx+1])

            # Обновляем веса
            w_new = self.w - self.eta * grad

            # Проверка критерия останова по норме разности весов
            if np.linalg.norm(w_new - self.w) < self.tolerance:
                self.w = w_new
                break
            self.w = w_new

        return self

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def predict_proba(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return self.sigmoid(X @ self.w)

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return (self.predict_proba(X) >= 0.5).astype(int)

    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        # Градиент LogLoss в векторном виде: (1/ell) * X^T * (sigmoid(Xw) - y)
        ell = X.shape[0]
        p = self.sigmoid(X @ self.w)
        grad = (X.T @ (p - y)) / ell
        return grad

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float
        """
        eps = 1e-15
        p = self.sigmoid(X @ self.w)
        loss = -np.mean(y * np.log(p + eps) + (1 - y) * np.log(1 - p + eps))
        return loss

Теперь проверим работу вашего класса на синтетических данных.

In [ ]:
from sklearn.preprocessing import StandardScaler

X, y = make_classification(
    n_samples=100000, n_features=20, n_informative=2,
    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### **Задание 2 (0.6 балла)**

Теперь давайте тестировать модель.
1. Обучите свою логистическую регрессию на синтетических данных (0.2 балла) — на полном GD и SGD;
2. Cравните результат с моделью из библиотеки. Посчитайте roc-auc, accuracy, постройте ROC и PR кривые. , оцените разницу в производительности моделей по метрикам качества. Ответьте на вопросы:
- Какая показывает лучший результат? Почему?
- Есть ли что-то в модели из коробки, что по умолчанию делает её не равной вашей модели? Для ответа на этот вопрос вам может пригодитться [документация](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Её мы изучаем всегда, чтобы понимать тонкости реализации какого-либо метода в библиотеке. (0.4 балла)

In [ ]:
lr_sgd = LogReg(gd_type='stochastic')
lr_sgd.fit(X_train, y_train)

lr_full = LogReg(gd_type='full')
lr_full.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt

# Обучаем логистическую регрессию на синтетических данных
lr_sklearn = LogisticRegression(max_iter=1000)
lr_sklearn.fit(X_train, y_train)

# Получаем предсказания
models = {
    'SGD (custom)': lr_sgd,
    'Full GD (custom)': lr_full,
    'sklearn LogReg': lr_sklearn,
}

# Считаем метрики
print("=" * 60)
print(f"{'Модель':<20} {'Accuracy':<12} {'ROC-AUC':<12}")
print("=" * 60)

for name, model in models.items():
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    # sklearn возвращает 2D массив для predict_proba
    if y_proba.ndim == 2:
        y_proba = y_proba[:, 1]

    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)
    print(f"{name:<20} {acc:<12.4f} {roc:<12.4f}")

print("=" * 60)

# Построение ROC и PR кривых
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC curves
ax1 = axes[0]
for name, model in models.items():
    y_proba = model.predict_proba(X_test)
    if y_proba.ndim == 2:
        y_proba = y_proba[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    ax1.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.4f})')

ax1.plot([0, 1], [0, 1], 'k--', label='Random')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('ROC Curve')
ax1.legend(loc='lower right')
ax1.grid(True)

# PR curves
ax2 = axes[1]
for name, model in models.items():
    y_proba = model.predict_proba(X_test)
    if y_proba.ndim == 2:
        y_proba = y_proba[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    pr_auc = auc(recall, precision)
    ax2.plot(recall, precision, label=f'{name} (AUC = {pr_auc:.4f})')

ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall Curve')
ax2.legend(loc='lower left')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Визуализация сходимости loss
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(lr_sgd.loss_history)
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss')
axes[0].set_title('SGD Loss History')
axes[0].grid(True)

axes[1].plot(lr_full.loss_history)
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Loss')
axes[1].set_title('Full GD Loss History')
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("Ответы на вопросы:")
print("=" * 60)
print("""
1. Какая модель показывает лучший результат? Почему?

По метрикам качества (Accuracy, ROC-AUC, PR-AUC) все три модели показывают очень близкие результаты.
Модель из библиотеки sklearn демонстрирует чуть более стабильное и иногда немного лучшее качество, однако различия не являются принципиальными.

Это объясняется тем, что логистическая регрессия с L2-регуляризацией — выпуклая задача, и при корректной настройке оптимизации как Full GD, так и SGD сходятся к близкому минимуму.
При этом sklearn LogisticRegression по умолчанию использует оптимизатор L-BFGS — квази-Ньютоновский метод, учитывающий информацию о кривизне функции потерь, что обеспечивает более быструю и устойчивую сходимость по сравнению с обычным GD и особенно с SGD.

Full GD сходится стабильно, но медленно, так как использует весь датасет на каждой итерации.
SGD обновляет веса по мини-батчам, из-за чего траектория оптимизации зашумлена и итоговое качество сильнее зависит от выбора learning rate.

2. Что делает sklearn модель не равной нашей?

Модель sklearn LogisticRegression не эквивалентна нашей,потому что по умолчанию включает ряд важных деталей:

- Регуляризация:
В sklearn по умолчанию используется L2-регуляризация (penalty='l2', C=1.0), тогда как в реализованной модели минимизируется чистый logloss без регуляризации.
- Интерцепт (bias):
В sklearn интерцепт добавляется автоматически (fit_intercept=True), а у нас свободный член отсутствует.
- Оптимизатор:
В sklearn по умолчанию применяется solver L-BFGS — квази-Ньютоновский метод, использующий информацию о кривизне функции потерь.
У нас реализованы только методы Full Gradient Descent и Stochastic Gradient Descent (по одному объекту).
- Критерии остановки:
В обеих реализациях присутствует ограничение по max_iter, однако критерий ранней остановки различается:
у нас  — по норме изменения весов ||w_new - w|| < tolerance,
в sklearn — по параметру tol, определяемому внутренними условиями сходимости выбранного solver’а.
-Стохастичность и воспроизводимость:
В реализации SGD используется случайный выбор объекта без фиксированного random_state, что может приводить к вариативности результата, тогда как в sklearn поведение можно контролировать через random_state (для соответствующих solver’ов).
""")

### **Задание 3 (0.4 балла)**

Для трех полученных моделей, визуализируйте прогнозы по данным на тестовой выборке. Для этого:
- получите прогнозы;
- сомжите данные, используя PCA. Не забудьте, что PCA полагает нулевое среднее и единичную дисперсию;
- покрасьте данные по прогнозам.

Как различаются графики для трёх моделей? И различаются ли?

In [ ]:
# Применяем PCA для визуализации (данные уже стандартизированы)
pca = PCA(n_components=2)
X_test_pca = pca.fit_transform(X_test)

# Получаем прогнозы от всех моделей
predictions = {
    'SGD (custom)': lr_sgd.predict(X_test),
    'Full GD (custom)': lr_full.predict(X_test),
    'sklearn LogReg': lr_sklearn.predict(X_test)
}

# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, (name, y_pred) in zip(axes, predictions.items()):
    scatter = ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1],
                         c=y_pred, cmap='coolwarm', alpha=0.5, s=10)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_title(f'{name}')
    ax.legend(*scatter.legend_elements(), title='Класс')

plt.tight_layout()
plt.show()

# Сравнение с истинными метками
fig, ax = plt.subplots(figsize=(6, 5))
scatter = ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1],
                     c=y_test, cmap='coolwarm', alpha=0.5, s=10)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('Истинные метки (y_test)')
ax.legend(*scatter.legend_elements(), title='Класс')
plt.show()

# Подсчет различий между моделями
print("=" * 50)
print("Совпадение прогнозов между моделями:")
print("=" * 50)
print(f"SGD vs Full GD:     {(predictions['SGD (custom)'] == predictions['Full GD (custom)']).mean():.4f}")
print(f"SGD vs sklearn:     {(predictions['SGD (custom)'] == predictions['sklearn LogReg']).mean():.4f}")
print(f"Full GD vs sklearn: {(predictions['Full GD (custom)'] == predictions['sklearn LogReg']).mean():.4f}")

print("\n" + "=" * 50)
print("ВЫВОД:")
print("=" * 50)
print("""
Графики практически не различаются. Все три модели формируют очень похожие области классификации в проекции PCA, а различия носят минимальный характер и сосредоточены вблизи границы между классами. Так, визуально границы решений у моделей отличаются: у SGD граница более размытая, у Full GD — наиболее четкая и стабильная, а у sklearn LogisticRegression — промежуточная, что связано с использованием L-BFGS и L2-регуляризации. При этом различия затрагивают лишь область вблизи границы классов и практически не влияют на итоговые метрики качества.

Причины схожести:
-Все три модели — линейные классификаторы.
-Задача выпуклая, и при достаточном числе итераций GD и SGD сходятся к близкому минимуму.
-Данные синтетические и хорошо структурированы, поэтому нет сложной нелинейной границы.
-PCA — это 2D-проекция, которая может сглаживать небольшие различия, существующие в исходном пространстве.

Небольшие различия могут быть вызваны из-за:
-стохастичности SGD (обновления по одному объекту);
-разных критериев остановки;
-наличия L2-регуляризации в sklearn;
-различия в оптимизаторе (L-BFGS vs GD/SGD).

""")

# **Часть 2. Обучение моделей на текстовых данных. (5 баллов)**

 ### **Подготовка данных из реального мира.**

Ещё одна прелесть простых моделей — возможность решать с ними неструктурированные (изначально не табличные) задачи. Давайте посмотрим на это в действии на примере текстов.


Загрузите данные с конкурса  [Natural Language Processing with Disaster Tweets](https://www.kaggle.com/competitions/nlp-getting-started/data?select=train.csv) (вам нужна только обучающая выборка, файл `train.csv`). Задача состоит в определении постов, сообщающих о чрезвычайной ситуации. В рамках домашнего задания, этот набор данных будет отличным полем для тренировки в обработке признаков.

In [ ]:
from sklearn.model_selection import train_test_split

PATH = 'train.csv'
data = pd.read_csv(PATH)

data.head()

### **Задание 10. Базовая предобработка (1.5 балла).**

- Выведите на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их пустой строкой (0.2 балла).

In [ ]:
# Анализ пропусков
print("Информация о пропусках в данных:")
print(data.isnull().sum())
print("\nПроцент пропусков:")
print(data.isnull().sum() / len(data) * 100)

# Заполнение пропусков пустой строкой
data['keyword'] = data['keyword'].fillna('')
data['location'] = data['location'].fillna('')
data['text'] = data['text'].fillna('')

print("\nПосле заполнения пропусков:")
print(data.isnull().sum())

- Проанализируйте количество уникальных значений в столбцах, опустив `text`. Сделайте выводы. (0.5 балла)

In [ ]:
# Анализ уникальных значений (без столбца text)
print("Количество уникальных значений в столбцах:")
for col in data.columns:
    if col != 'text':
        print(f"{col}: {data[col].nunique()}")

print("\n" + "="*60)
print("ВЫВОДЫ:")
print("="*60)
print("1. id: {} уникальных значений - это идентификаторы твитов".format(data['id'].nunique()))
print("2. keyword: {} уникальных значений - ключевые слова из твитов".format(data['keyword'].nunique()))
print("   (с учетом пустых строк после заполнения пропусков)")
print("3. location: {} уникальных значений - местоположения авторов".format(data['location'].nunique()))
print("   (очень много разных локаций, что может указывать на шум в данных)")
print("4. target: {} класса - бинарная классификация (0 - не ЧС, 1 - ЧС)".format(data['target'].nunique()))

- Проанализируйте соотношение классов в целевой переменной. Какое оно? Выберите метрику, с помощью которой будете оценивать модель.  (0.5 балла)

In [ ]:
# Анализ баланса классов
print("Распределение классов:")
print(data['target'].value_counts())
print("\nПроцентное соотношение:")
print(data['target'].value_counts(normalize=True) * 100)

# Визуализация
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
data['target'].value_counts().plot(kind='bar', ax=ax)
ax.set_xlabel('Класс (0 - не ЧС, 1 - ЧС)')
ax.set_ylabel('Количество')
ax.set_title('Распределение классов в целевой переменной')
ax.set_xticklabels(['Не ЧС (0)', 'ЧС (1)'], rotation=0)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("ВЫВОД:")
print("="*60)
print("Соотношение классов примерно 57% : 43% (не ЧС : ЧС).")
print("Данные слегка несбалансированы, но не критично.")
print("\nВыбранная метрика: ROC-AUC")
print("Причина: ROC-AUC хорошо работает при небольшом дисбалансе классов")
print("и оценивает способность модели различать классы по вероятностям.")
print("Также будем использовать accuracy для дополнительной оценки.")

- Объедините все три текстовых столбца в один для baseline (вам поможет конкатенация строк) (0.3 балла)

In [ ]:
# Объединение текстовых столбцов для baseline
# Конкатенация keyword, location и text с пробелами
data_new = data.copy()
data_new['text'] = data['keyword'] + ' ' + data['location'] + ' ' + data['text']

print("Пример объединенного текста:")
print(data_new['text'].head(3))
print("\nФорма данных:", data_new.shape)

- Наконец, поделите данные на тренировочную и тестовую выборки. (0.2 балла)

In [ ]:
X = data_new['text']
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)
print("Распределение классов в обучающей выборке:")
print(y_train.value_counts())

### **Задание 11. Базовые модели. (1 балл).**

Данные, собираемые с сайтов, часто содержат мусор не информативный для моделей. Посмотрите, какого качества и насколько разнообразны данные здесь. Для этого:
- Примените CountVectorizer из sklearn к сырым даным. Какого размера получилась матрица? (0.3 балла)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Применяем CountVectorizer к сырым данным
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("Размер матрицы признаков (обучающая выборка):", X_train_vec.shape)
print("Размер матрицы признаков (тестовая выборка):", X_test_vec.shape)
print("\nКоличество объектов:", X_train_vec.shape[0])
print("Количество признаков (размер словаря):", X_train_vec.shape[1])
print("\nМатрица получилась очень разреженной с {} уникальными словами".format(X_train_vec.shape[1]))

- Обучите логистическую регрессию на полученном наборе. Модель возьмите из библиотеки. Какое качество по выбранной вами метрике у модели получилось на тестовых данных? (0.3 балла)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import time
import warnings

# Обучаем логистическую регрессию с улучшенными параметрами
# C=1.0 - добавляем регуляризацию для численной стабильности
# solver='liblinear' - более устойчивый solver для разреженных данных
lr = LogisticRegression(max_iter=1000, random_state=42, C=1.0, solver='liblinear')

start_time = time.time()
lr.fit(X_train_vec, y_train)
train_time = time.time() - start_time

# Предсказания
y_pred_lr = lr.predict(X_test_vec)
y_proba_lr = lr.predict_proba(X_test_vec)[:, 1]

# Метрики
acc_lr = accuracy_score(y_test, y_pred_lr)
roc_auc_lr = roc_auc_score(y_test, y_proba_lr)

print("="*60)
print("Логистическая регрессия (baseline)")
print("="*60)
print(f"Время обучения: {train_time:.2f} секунд")
print(f"Accuracy на тесте: {acc_lr:.4f}")
print(f"ROC-AUC на тесте: {roc_auc_lr:.4f}")
print("="*60)

In [ ]:
# Визуализация ROC-кривой для логистической регрессии
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(y_test, y_proba_lr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {roc_auc_lr:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend()
plt.grid(True)
plt.show()

- Обучите SVC на тех же данных с гиперпараметрами по умолчанию. Измерьте качество на тестовых данных и опишите результат. Проанализируйте качество и скорость обучения.(0.4 балла)

In [ ]:
from sklearn.svm import SVC
import time

# Обучаем SVC с гиперпараметрами по умолчанию
svc = SVC(random_state=42, probability=True)

start_time = time.time()
svc.fit(X_train_vec, y_train)
train_time_svc = time.time() - start_time

# Предсказания
y_pred_svc = svc.predict(X_test_vec)
y_proba_svc = svc.predict_proba(X_test_vec)[:, 1]

# Метрики
acc_svc = accuracy_score(y_test, y_pred_svc)
roc_auc_svc = roc_auc_score(y_test, y_proba_svc)

print("="*60)
print("SVC (baseline)")
print("="*60)
print(f"Время обучения: {train_time_svc:.2f} секунд")
print(f"Accuracy на тесте: {acc_svc:.4f}")
print(f"ROC-AUC на тесте: {roc_auc_svc:.4f}")
print("="*60)

print("\n" + "="*60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ:")
print("="*60)
print(f"Логистическая регрессия:")
print(f"  - Время обучения: {train_time:.2f} сек")
print(f"  - ROC-AUC: {roc_auc_lr:.4f}")
print(f"\nSVC:")
print(f"  - Время обучения: {train_time_svc:.2f} сек")
print(f"  - ROC-AUC: {roc_auc_svc:.4f}")
print(f"\nВЫВОД:")
print(f"SVC обучается значительно медленнее (в {train_time_svc/train_time:.1f} раз),")
print(f"но показывает {'лучшее' if roc_auc_svc > roc_auc_lr else 'аналогичное'} качество.")
print("Для больших датасетов логистическая регрессия предпочтительнее")
print("из-за скорости обучения при сопоставимом качестве.")

### **Задание 12. Улучшение базовых моделей за счет данных. (0.3 балла).**

- Подберите гиперпараметры CountVectorizer так, чтобы признаков было минимум в 4 раза меньше, чем объектов, а качество модели при этом изменилось не более чем на $\pm 0.07$. Опишите подобранные гиперпараметры и на что они влияют.

Обучайте и логистическую регрессию, и SVC.

In [ ]:
# Подбор гиперпараметров CountVectorizer
# Цель: признаков минимум в 4 раза меньше объектов (5329 / 4 = 1332)
# при этом качество должно измениться не более чем на ±0.07

# Используем следующие гиперпараметры:
# - max_features: ограничиваем количество признаков
# - min_df: минимальная частота документов для слова
# - max_df: максимальная частота документов (убираем очень частые слова)
# - ngram_range: используем униграммы и биграммы

vectorizer_improved = CountVectorizer(
    max_features=1200,  # Ограничиваем количество признаков
    min_df=2,           # Слово должно встречаться минимум в 2 документах
    max_df=0.8,         # Убираем слова, встречающиеся более чем в 80% документов
    ngram_range=(1, 2)  # Используем униграммы и биграммы
)

X_train_vec_improved = vectorizer_improved.fit_transform(X_train)
X_test_vec_improved = vectorizer_improved.transform(X_test)

print("="*60)
print("Улучшенный CountVectorizer")
print("="*60)
print(f"Количество объектов: {X_train_vec_improved.shape[0]}")
print(f"Количество признаков: {X_train_vec_improved.shape[1]}")
print(f"Соотношение объекты/признаки: {X_train_vec_improved.shape[0] / X_train_vec_improved.shape[1]:.2f}")
print("="*60)

print("\nОписание подобранных гиперпараметров:")
print("1. max_features=1200 - ограничивает словарь 1200 наиболее частыми словами")
print("   Влияние: уменьшает размерность, убирает редкие/шумные признаки")
print("\n2. min_df=2 - слово должно встретиться минимум в 2 документах")
print("   Влияние: фильтрует опечатки и очень редкие слова")
print("\n3. max_df=0.8 - исключает слова, встречающиеся в >80% документов")
print("   Влияние: убирает стоп-слова (the, is, a, etc.)")
print("\n4. ngram_range=(1,2) - использует униграммы и биграммы")
print("   Влияние: учитывает контекст слов (например, 'not good' vs 'good')")

In [ ]:
# Обучаем LogisticRegression на улучшенных признаках
# Используем solver='liblinear' для численной стабильности
lr_improved = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
lr_improved.fit(X_train_vec_improved, y_train)

y_pred_lr_improved = lr_improved.predict(X_test_vec_improved)
y_proba_lr_improved = lr_improved.predict_proba(X_test_vec_improved)[:, 1]

acc_lr_improved = accuracy_score(y_test, y_pred_lr_improved)
roc_auc_lr_improved = roc_auc_score(y_test, y_proba_lr_improved)

# Обучаем SVC на улучшенных признаках
svc_improved = SVC(random_state=42, probability=True)
svc_improved.fit(X_train_vec_improved, y_train)

y_pred_svc_improved = svc_improved.predict(X_test_vec_improved)
y_proba_svc_improved = svc_improved.predict_proba(X_test_vec_improved)[:, 1]

acc_svc_improved = accuracy_score(y_test, y_pred_svc_improved)
roc_auc_svc_improved = roc_auc_score(y_test, y_proba_svc_improved)

# Сравнение результатов
print("="*60)
print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ")
print("="*60)
print(f"{'Модель':<30} {'Baseline ROC-AUC':<20} {'Improved ROC-AUC':<20} {'Изменение':<15}")
print("-"*60)
print(f"{'Logistic Regression':<30} {roc_auc_lr:<20.4f} {roc_auc_lr_improved:<20.4f} {roc_auc_lr_improved - roc_auc_lr:+.4f}")
print(f"{'SVC':<30} {roc_auc_svc:<20.4f} {roc_auc_svc_improved:<20.4f} {roc_auc_svc_improved - roc_auc_svc:+.4f}")
print("="*60)

# Проверка условий задания
change_lr = abs(roc_auc_lr_improved - roc_auc_lr)
change_svc = abs(roc_auc_svc_improved - roc_auc_svc)

print(f"\nУсловие 1: Признаков в 4+ раза меньше объектов")
print(f"  {X_train_vec_improved.shape[0]} / {X_train_vec_improved.shape[1]} = {X_train_vec_improved.shape[0] / X_train_vec_improved.shape[1]:.2f} ✓")
print(f"\nУсловие 2: Качество изменилось не более чем на ±0.07")
print(f"  LogReg: {change_lr:.4f} {'✓' if change_lr <= 0.07 else '✗'}")
print(f"  SVC: {change_svc:.4f} {'✓' if change_svc <= 0.07 else '✗'}")

### **Задание 13. Улучшение базовых моделей за счет данных 2. (0.7 балла).**

В первом пункте мы склеили все строки в одну. Но можно было бы поступить иначе — и получить категории из `'keyword', 'location'`. Протестируйте такой подход на обеих моделях и замерьте качество. Улучшает ли это результат?

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import numpy as np

# Вместо конкатенации, обрабатываем текст отдельно, а keyword и location как категории
# Векторизуем только текст
vectorizer_text = CountVectorizer(max_features=1200, min_df=2, max_df=0.8, ngram_range=(1, 2))

# Подготовка данных
X_train_text = data.loc[X_train.index, 'text'].fillna('')
X_test_text = data.loc[X_test.index, 'text'].fillna('')

X_train_text_vec = vectorizer_text.fit_transform(X_train_text)
X_test_text_vec = vectorizer_text.transform(X_test_text)

# Категориальные признаки: keyword и location
X_train_keyword = data.loc[X_train.index, 'keyword'].fillna('').values.reshape(-1, 1)
X_test_keyword = data.loc[X_test.index, 'keyword'].fillna('').values.reshape(-1, 1)

X_train_location = data.loc[X_train.index, 'location'].fillna('').values.reshape(-1, 1)
X_test_location = data.loc[X_test.index, 'location'].fillna('').values.reshape(-1, 1)

# One-hot encoding для категорий
encoder_keyword = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
encoder_location = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

X_train_keyword_enc = encoder_keyword.fit_transform(X_train_keyword)
X_test_keyword_enc = encoder_keyword.transform(X_test_keyword)

X_train_location_enc = encoder_location.fit_transform(X_train_location)
X_test_location_enc = encoder_location.transform(X_test_location)

# Объединяем все признаки
X_train_combined = hstack([X_train_text_vec, X_train_keyword_enc, X_train_location_enc])
X_test_combined = hstack([X_test_text_vec, X_test_keyword_enc, X_test_location_enc])

print("="*60)
print("Признаки с категориями")
print("="*60)
print(f"Размерность текстовых признаков: {X_train_text_vec.shape[1]}")
print(f"Размерность keyword признаков: {X_train_keyword_enc.shape[1]}")
print(f"Размерность location признаков: {X_train_location_enc.shape[1]}")
print(f"Итоговая размерность: {X_train_combined.shape[1]}")
print("="*60)

In [ ]:
# Обучаем модели на признаках с категориями (X_train_combined)
print("Обучение моделей на признаках с категориями...")
print("="*60)

# LogisticRegression с категориями
lr_categories = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
lr_categories.fit(X_train_combined, y_train)

y_pred_lr_cat = lr_categories.predict(X_test_combined)
y_proba_lr_cat = lr_categories.predict_proba(X_test_combined)[:, 1]

acc_lr_cat = accuracy_score(y_test, y_pred_lr_cat)
roc_auc_lr_cat = roc_auc_score(y_test, y_proba_lr_cat)

# SVC с категориями
svc_categories = SVC(random_state=42, probability=True)
svc_categories.fit(X_train_combined, y_train)

y_pred_svc_cat = svc_categories.predict(X_test_combined)
y_proba_svc_cat = svc_categories.predict_proba(X_test_combined)[:, 1]

acc_svc_cat = accuracy_score(y_test, y_pred_svc_cat)
roc_auc_svc_cat = roc_auc_score(y_test, y_proba_svc_cat)

# Сравнение с baseline (из задания 12 - улучшенный vectorizer)
print("\nСРАВНЕНИЕ ПОДХОДОВ:")
print("="*60)
print(f"{'Модель':<30} {'Конкатенация':<20} {'Категории':<20} {'Разница':<15}")
print("-"*60)
print(f"{'Logistic Regression':<30} {roc_auc_lr_improved:<20.4f} {roc_auc_lr_cat:<20.4f} {roc_auc_lr_cat - roc_auc_lr_improved:+.4f}")
print(f"{'SVC':<30} {roc_auc_svc_improved:<20.4f} {roc_auc_svc_cat:<20.4f} {roc_auc_svc_cat - roc_auc_svc_improved:+.4f}")
print("="*60)

# Ответ на вопрос задания
print("\n" + "="*60)
print("ОТВЕТ НА ВОПРОС: Улучшает ли использование категорий результат?")
print("="*60)

if roc_auc_lr_cat > roc_auc_lr_improved and roc_auc_svc_cat > roc_auc_svc_improved:
    print("ДА, использование категорий улучшает качество обеих моделей!")
    print(f"LogReg: улучшение на {(roc_auc_lr_cat - roc_auc_lr_improved)*100:.2f}%")
    print(f"SVC: улучшение на {(roc_auc_svc_cat - roc_auc_svc_improved)*100:.2f}%")
elif roc_auc_lr_cat < roc_auc_lr_improved or roc_auc_svc_cat < roc_auc_svc_improved:
    print("НЕТ, использование категорий НЕ улучшает результат.")
    print(f"LogReg: изменение {(roc_auc_lr_cat - roc_auc_lr_improved)*100:+.2f}%")
    print(f"SVC: изменение {(roc_auc_svc_cat - roc_auc_svc_improved)*100:+.2f}%")
    print("\nВОЗМОЖНЫЕ ПРИЧИНЫ:")
    print("- Location содержит много шума (2471 уникальных значений)")
    print("- Конкатенация текста уже включает информацию из keyword и location")
    print("- Модель переобучается на разреженных категориальных признаках")
else:
    print("РЕЗУЛЬТАТЫ СМЕШАННЫЕ - одна модель улучшилась, другая ухудшилась")

print(f"\nИтоговая размерность с категориями: {X_train_combined.shape[1]} признаков")
print(f"Размерность с конкатенацией: {X_train_vec_improved.shape[1]} признаков")
print("="*60)

### **Задание 13. Улучшение базовых моделей путем подбора гиперпараметров. (1 балл).**
- Попробуйте подбирать разные гиперпараметры для логистической регрессии. Опишите подбираемые гиперапарметры и ваши результаты (0.5 балла)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Подбор гиперпараметров для LogisticRegression
# Основные гиперпараметры:
# - C: обратная величина регуляризации (меньше C = сильнее регуляризация)
# - penalty: тип регуляризации (l1, l2, elasticnet)
# - solver: алгоритм оптимизации

# Используем только liblinear для стабильности и избежания warnings
param_grid_lr = {
    'C': [0.1, 1, 10, 100],                    # Сила регуляризации
    'penalty': ['l1', 'l2'],                   # Тип регуляризации
    'solver': ['liblinear']                    # liblinear работает с l1 и l2, устойчив к численным проблемам
}

lr_grid = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=42),
    param_grid_lr,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

print("Подбор гиперпараметров для LogisticRegression...")
lr_grid.fit(X_train_combined, y_train)

print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ПОДБОРА ГИПЕРПАРАМЕТРОВ - LogisticRegression")
print("="*60)
print(f"Лучшие параметры: {lr_grid.best_params_}")
print(f"Лучший ROC-AUC (CV): {lr_grid.best_score_:.4f}")

# Тестируем на тестовой выборке
y_proba_lr_tuned = lr_grid.best_estimator_.predict_proba(X_test_combined)[:, 1]
roc_auc_lr_tuned = roc_auc_score(y_test, y_proba_lr_tuned)
print(f"ROC-AUC на тесте: {roc_auc_lr_tuned:.4f}")
print(f"Улучшение по сравнению с baseline: {roc_auc_lr_tuned - roc_auc_lr:+.4f}")
print("="*60)

print("\nОПИСАНИЕ ГИПЕРПАРАМЕТРОВ:")
print("1. C - обратная сила регуляризации:")
print("   - Меньшие значения (0.1) = сильная регуляризация = проще модель")
print("   - Большие значения (100) = слабая регуляризация = сложнее модель")
print("\n2. penalty - тип регуляризации:")
print("   - 'l2' (Ridge): штрафует квадрат весов, делает их маленькими")
print("   - 'l1' (Lasso): может обнулять веса, выполняет отбор признаков")
print("\n3. solver - алгоритм оптимизации:")
print("   - 'liblinear': оптимален для малых/средних данных, работает с l1 и l2,")
print("     устойчив к численным проблемам с разреженными матрицами")

In [ ]:
# Подбор гиперпараметров для SVC
# Основные гиперпараметры:
# - C: параметр регуляризации
# - kernel: тип ядра (linear, rbf, poly)
# - gamma: коэффициент ядра для rbf, poly

# Используем меньший grid для SVC, так как он обучается медленно
param_grid_svc = {
    'C': [0.1, 1, 10],                    # Сила регуляризации
    'kernel': ['linear', 'rbf'],          # Тип ядра
    'gamma': ['scale', 'auto']            # Коэффициент ядра
}

svc_grid = GridSearchCV(
    SVC(random_state=42, probability=True),
    param_grid_svc,
    cv=3,  # Меньше фолдов для ускорения
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

print("Подбор гиперпараметров для SVC...")
print("(это может занять несколько минут...)")
svc_grid.fit(X_train_combined, y_train)

print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ПОДБОРА ГИПЕРПАРАМЕТРОВ - SVC")
print("="*60)
print(f"Лучшие параметры: {svc_grid.best_params_}")
print(f"Лучший ROC-AUC (CV): {svc_grid.best_score_:.4f}")

# Тестируем на тестовой выборке
y_proba_svc_tuned = svc_grid.best_estimator_.predict_proba(X_test_combined)[:, 1]
roc_auc_svc_tuned = roc_auc_score(y_test, y_proba_svc_tuned)
print(f"ROC-AUC на тесте: {roc_auc_svc_tuned:.4f}")
print(f"Улучшение по сравнению с baseline: {roc_auc_svc_tuned - roc_auc_svc:+.4f}")
print("="*60)

print("\nОПИСАНИЕ ГИПЕРПАРАМЕТРОВ:")
print("1. C - параметр регуляризации:")
print("   - Меньшие значения (0.1) = широкая разделяющая полоса = проще модель")
print("   - Большие значения (10) = узкая разделяющая полоса = сложнее модель")
print("\n2. kernel - тип ядра:")
print("   - 'linear': линейное разделение, быстрое обучение")
print("   - 'rbf': нелинейное разделение (радиальная базисная функция)")
print("\n3. gamma - коэффициент ядра (для rbf):")
print("   - 'scale': автоматический расчет на основе данных (рекомендуется)")
print("   - 'auto': 1 / n_features")
print("   - Большие gamma = учитывается только ближайшее окружение точки")

**ОБЩИЕ ВЫВОДЫ ПО ПОДБОРУ ГИПЕРПАРАМЕТРОВ:**

1. Подбор гиперпараметров позволяет значительно улучшить качество моделей
2. LogisticRegression обучается быстрее и хорошо подходит для текстовых данных
3. SVC может показывать лучшее качество, но требует больше времени на обучение
4. Использование GridSearchCV с кросс-валидацией помогает избежать переобучения
5. Важно балансировать между сложностью модели и обобщающей способностью через регуляризацию

Ваши выводы здесь.

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

# Создаем функции для извлечения признаков
def get_text(df):
    """Извлекает текстовый столбец"""
    if isinstance(df, pd.Series):
        return df
    return df['text'] if 'text' in df.columns else df

def get_keyword(df):
    """Извлекает keyword столбец"""
    if isinstance(df, pd.Series):
        return pd.DataFrame({'keyword': [''] * len(df)})
    return df[['keyword']].fillna('')

def get_location(df):
    """Извлекает location столбец"""
    if isinstance(df, pd.Series):
        return pd.DataFrame({'location': [''] * len(df)})
    return df[['location']].fillna('')

# Пайплайн для LogisticRegression
pipeline_lr = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('selector', FunctionTransformer(lambda x: x if isinstance(x, pd.Series) else x['text'].fillna(''), validate=False)),
            ('vectorizer', CountVectorizer(max_features=1200, min_df=2, max_df=0.8, ngram_range=(1, 2)))
        ])),
        ('keyword', Pipeline([
            ('selector', FunctionTransformer(lambda x: x[['keyword']].fillna('') if isinstance(x, pd.DataFrame) else pd.DataFrame({'keyword': ['']*len(x)}), validate=False)),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
        ])),
        ('location', Pipeline([
            ('selector', FunctionTransformer(lambda x: x[['location']].fillna('') if isinstance(x, pd.DataFrame) else pd.DataFrame({'location': ['']*len(x)}), validate=False)),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
        ]))
    ])),
    ('classifier', LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear'))
])

# Пайплайн для SVC
pipeline_svc = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('selector', FunctionTransformer(lambda x: x if isinstance(x, pd.Series) else x['text'].fillna(''), validate=False)),
            ('vectorizer', CountVectorizer(max_features=1200, min_df=2, max_df=0.8, ngram_range=(1, 2)))
        ])),
        ('keyword', Pipeline([
            ('selector', FunctionTransformer(lambda x: x[['keyword']].fillna('') if isinstance(x, pd.DataFrame) else pd.DataFrame({'keyword': ['']*len(x)}), validate=False)),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
        ])),
        ('location', Pipeline([
            ('selector', FunctionTransformer(lambda x: x[['location']].fillna('') if isinstance(x, pd.DataFrame) else pd.DataFrame({'location': ['']*len(x)}), validate=False)),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
        ]))
    ])),
    ('classifier', SVC(C=10, kernel='rbf', gamma='scale', probability=True, random_state=42))
])

print("="*60)
print("СОЗДАНИЕ И ТЕСТИРОВАНИЕ ПАЙПЛАЙНОВ")
print("="*60)

# Подготовка данных для пайплайнов (используем исходный DataFrame)
X_train_df = data.loc[X_train.index, ['text', 'keyword', 'location']]
X_test_df = data.loc[X_test.index, ['text', 'keyword', 'location']]

# Обучение и тестирование пайплайна LogisticRegression
print("\nОбучение пайплайна LogisticRegression...")
pipeline_lr.fit(X_train_df, y_train)
y_proba_pipeline_lr = pipeline_lr.predict_proba(X_test_df)[:, 1]
roc_auc_pipeline_lr = roc_auc_score(y_test, y_proba_pipeline_lr)

print(f"ROC-AUC (LogisticRegression Pipeline): {roc_auc_pipeline_lr:.4f}")

# Обучение и тестирование пайплайна SVC
print("\nОбучение пайплайна SVC...")
pipeline_svc.fit(X_train_df, y_train)
y_proba_pipeline_svc = pipeline_svc.predict_proba(X_test_df)[:, 1]
roc_auc_pipeline_svc = roc_auc_score(y_test, y_proba_pipeline_svc)

print(f"ROC-AUC (SVC Pipeline): {roc_auc_pipeline_svc:.4f}")

print("\n" + "="*60)
print("СТРУКТУРА ПАЙПЛАЙНОВ:")
print("="*60)
print("\nОба пайплайна включают:")
print("1. Feature extraction:")
print("   - Text: CountVectorizer с оптимизированными параметрами")
print("   - Keyword: OneHotEncoder для категориальных признаков")
print("   - Location: OneHotEncoder для категориальных признаков")
print("2. Feature union: объединение всех признаков")
print("3. Classifier: LogisticRegression или SVC с настроенными параметрами")
print("\nПреимущества пайплайна:")
print("- Автоматическая предобработка новых данных")
print("- Предотвращение утечки данных между train и test")
print("- Удобство в продакшене")
print("- Простота в использовании с GridSearchCV")
print("="*60)

**Опишите общие мысли о работе. Это место для вашей рефлексии, не обязательное, но полезное.**  🐤